<a href="https://colab.research.google.com/github/AAKAAASSHHH24/Data_Extraction_And_NLP/blob/main/Data_Extraction_and_NLP_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
ROOT = "/content/drive/MyDrive/BlackCoffer"
os.chdir(ROOT)

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
# Using Selenium to extract contents from the websites
!pip install selenium
from selenium.webdriver.chrome.service import Service
!pip install webdriver_manager

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached urllib3-1.26.13-py2.py3-none-any.whl (140 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.11
    Uninstalling urllib3-1.25.11:
      Successfully uninstalled urllib3-1.25.11
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.13 which is incompatible.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached urllib3-1.25.11-py2.py3-none-any.whl (127 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.13
    Uninstalling urllib3-1.26.13:
      Successfully uninstalled urllib3-1.26.13
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
selenium 4.7.2 requires urllib3[socks]~=1.26, but you have urllib3 1.25.11 which is incompatible.


In [ ]:
# We will use BeautifulSoup to read the html contents

In [ ]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import codecs
import re
from webdriver_manager.chrome import ChromeDriverManager
#from selenium.webdriver.chrome.service import Service

/usr/local/lib/python3.8/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [ ]:
!apt-get update 
!apt install chromium-chromedriver

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
#driver=webdriver.Chrome(service=Service(ChromeDriverManager().install()))

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:13 http://ppa.launchpad.net

<ipython-input-6-52d3d6fb019c>:9: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)


In [ ]:
!pwd

/content/drive/MyDrive/BlackCoffer


In [ ]:
with open('Input.xlsx', 'rb') as f:
        data = pd.read_excel(f)
data

,URL_ID,URL
0,37.0,https://insights.blackcoffer.com/ai-in-healthc...
1,38.0,https://insights.blackcoffer.com/what-if-the-c...
2,39.0,https://insights.blackcoffer.com/what-jobs-wil...
3,40.0,https://insights.blackcoffer.com/will-machine-...
4,41.0,https://insights.blackcoffer.com/will-ai-repla...
...,...,...
109,146.0,https://insights.blackcoffer.com/blockchain-fo...
110,147.0,https://insights.blackcoffer.com/the-future-of...
111,148.0,https://insights.blackcoffer.com/big-data-anal...
112,149.0,https://insights.blackcoffer.com/business-anal...


In [ ]:
# function to get the source of the webpage
def get_page_source(web):
  wait = WebDriverWait(driver, 10)
  driver.get(web)
  get_url = driver.current_url
  wait.until(EC.url_to_be(web))


  if get_url == web:
    page_source = driver.page_source
    return page_source

In [ ]:
# function to get texr from the webpage
def get_text(page_source):
  soup = BeautifulSoup(page_source,"html.parser")

  text = soup.find_all('p')

  return text

In [ ]:
# function to filter out the unwanted text from the text
def preprocess_content(text):
  content = []
  paras = len(text)
  for i in range(paras):
    soup2 = BeautifulSoup(str(text[i]), 'html.parser')
    content.append(re.sub('\xa0', '', soup2.get_text()))
  return content

In [ ]:
# Create a directory for the article files if it doesn't already exist
def create_directory():
  if not os.path.exists("articles"):
      os.makedirs("articles")

In [ ]:
# Save the article text to a file
def save_text_file(text,content,article_id):
  paras = len(text)
  if not os.path.exists(f"articles/{article_id}.txt"):
    with open(f"articles/{article_id}.txt", "w") as f:
      for i in range(paras):
        f.write(content[i])

In [ ]:
from typing_extensions import dataclass_transform
def run_processes():
  for index, row in data.iterrows():
    # Access data for each column by column name
    article_id = data['URL_ID'][index]
    WEB =data['URL'][index]
    page_source = get_page_source(WEB)
    text = get_text(page_source)
    content = preprocess_content(text)
    create_directory()
    save_text_file(text,content,article_id)

In [ ]:
run_processes()

**TEXT ANALYSIS**

In [ ]:
os.listdir()

['Input.xlsx',
 'StopWords-20221226T201449Z-001.zip',
 'MasterDictionary-20221226T201605Z-001.zip',
 'articles',
 'StopWords',
 'MasterDictionary']

In [ ]:
#!unzip StopWords-20221226T201449Z-001.zip

Archive:  StopWords-20221226T201449Z-001.zip
  inflating: StopWords/StopWords_Geographic.txt  
  inflating: StopWords/StopWords_Currencies.txt  
  inflating: StopWords/StopWords_GenericLong.txt  
  inflating: StopWords/StopWords_Generic.txt  
  inflating: StopWords/StopWords_Auditor.txt  
  inflating: StopWords/StopWords_Names.txt  
  inflating: StopWords/StopWords_DatesandNumbers.txt  


In [ ]:
#!unzip MasterDictionary-20221226T201605Z-001.zip

Archive:  MasterDictionary-20221226T201605Z-001.zip
  inflating: MasterDictionary/negative-words.txt  
  inflating: MasterDictionary/positive-words.txt  


In [ ]:
os.chdir('/content/drive/MyDrive/BlackCoffer')

In [ ]:
os.chdir('StopWords')
stopwords_type = os.listdir()
stopwords_type

['StopWords_Names.txt',
 'StopWords_Geographic.txt',
 'StopWords_GenericLong.txt',
 'StopWords_DatesandNumbers.txt',
 'StopWords_Currencies.txt',
 'StopWords_Auditor.txt',
 'StopWords_Generic.txt']

In [ ]:
import chardet
stop_words = []
for i in stopwords_type:
  #print('start')
  with open(i , "rb") as f:
    contents = f.read()
    encoding = chardet.detect(contents)["encoding"]
  with open(i , "r", encoding = encoding) as f:
    l =   f.read().split()
  stop_words = stop_words+l
stop_words

In [ ]:
# nltk
import nltk
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer
import re

In [ ]:
nltk.download('stopwords')
stopwords = stopwords.words("english")
stopwords

In [ ]:
print(len(stopwords))
print(len(stop_words))
stopwords = stopwords + stop_words
print(len(stopwords))

179
14336
14515


In [ ]:
CLEAN_TEXT = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"
stemmer = SnowballStemmer("english")
def preprocess(text, stem=False):
    # Remove link,user and special characters
    text = re.sub(CLEAN_TEXT, ' ', str(text).lower()).strip()
    filtered_words = [word for word in text.split() if word.lower() not in stopwords]
    tokens = []
    for token in filtered_words:
        if token not in stopwords:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return " ".join(tokens)

In [ ]:
os.chdir('/content/drive/MyDrive/BlackCoffer/articles')

In [ ]:
# read content from the text file
def read_para(text_file):
  with open( text_file, "rb") as f:
   contents = f.read()
  return contents

In [ ]:
# get contents of each text file in the form of dictionary
def get_contents():
  os.chdir('/content/drive/MyDrive/BlackCoffer/articles')
  contents = {}
  for i in range(len(os.listdir())):
    contents[i] = read_para(os.listdir()[i])
    contents[i]  = str(contents[i] )
    contents[i] = contents[i].replace('\\xe2', ' ')
    contents[i]  = contents[i].replace('\\x80', ' ')
    contents[i]  = contents[i].replace('\\x99', ' ')
    contents[i]  = contents[i].replace('\\xa6', ' ')
    contents[i]  = contents[i].replace('\\x93', ' ')
    contents[i] = preprocess(contents[i])

    i+=1

  return contents

In [ ]:
# number of words in in text file in the form of a list
def number_of_words(contents):
  os.chdir('/content/drive/MyDrive/BlackCoffer/articles')
  num_words = []
  for i in range(len(os.listdir())):
    k = contents[i].split()
    num_words.append(len(k))
    i+=1
   
  return num_words

In [ ]:
# number of personal pronouns in each text file in the form of a list
def personal_pronouns():
  os.chdir('/content/drive/MyDrive/BlackCoffer/articles')
  contents = {}
  num_personal_pronouns=[]
  for i in range(len(os.listdir())):
    contents[i] = read_para(os.listdir()[i])
    contents[i] = str(contents[i])
    q = len([element for element in contents[i].split() if element in ["I","you"  ,                                                                                      "you" ,
                                                                                        "he" ,"she",
                                                                                        "it", "we", 
                                                                                        "they", 
                                                                                        "them", 
                                                                                        "us", 
                                                                                        "him", 
                                                                                        "her", 
                                                                                        "his", 
                                                                                        "hers", 
                                                                                        "its", 
                                                                                        "theirs", 
                                                                                        "our", "your"]])
    num_personal_pronouns.append(q)
    i+=1
  return num_personal_pronouns


In [ ]:
# number of sentences in each text file in the form of a list
def num_sentences():
  os.chdir('/content/drive/MyDrive/BlackCoffer/articles')
  contents = {}
  num_sentences =[]
  for i in range(len(os.listdir())):
    contents[i] = read_para(os.listdir()[i])
    contents[i] = str(contents[i]) 
    sentences = contents[i].split('. ') 
    sentences = [sentences[j] for j in range(len(sentences))]   
    num_sentences.append(len(sentences))

    i+=1

  return num_sentences  


In [ ]:
# read text file linewise
def read_text(filename):
    with open(filename, 'rb') as f:
        contents = f.readlines()
        #words = contents.split('\n')
    return contents


In [ ]:
# extract the list of positive and negative words from the master dictionary
def words_type():
  path = '/content/drive/MyDrive/BlackCoffer/MasterDictionary'
  os.chdir(path)
  positive_words, negative_words = read_text(os.listdir()[0]),read_text(os.listdir()[1])
  positive_words = [str(positive_words[i])[2:-3] for i in range(len(positive_words))]
  negative_words = [str(negative_words[i])[2:-3] for i in range(len(negative_words))]
  return positive_words,negative_words

In [ ]:
positive_words, negative_words = words_type()

## **Calculating Scores**

In [ ]:
# calculate positive and negative score of all articles, return in the form of a list
def score(list1, contents):
  score = []
  for i in range(len(contents)):
    common_elements =[]
    current = contents[i].split()
    common_elements= [element for element in list1 if element in current]
  
    score.append(len(common_elements))
    i+=1

  return score

In [ ]:
def get_positive_score(contents):
  positive_words, negative_words = words_type()
  positive_score = score(positive_words, contents)
  return positive_score

In [ ]:
def get_negative_score(contents):
  positive_words, negative_words = words_type()
  negative_score = score(negative_words, contents)
  return negative_score

In [ ]:
# polarity scores of all articles int he form of text
def polarity(positive_score, negative_score):
  os.chdir('/content/drive/MyDrive/BlackCoffer/articles')
  polarity = []
  for i in range(len(os.listdir())):
    polarity.append((positive_score[i]-negative_score[i])/((positive_score[i]+negative_score[i]) + .000001))
    i+=1
  
  return polarity

In [ ]:
# average sentence length of each text file in the form of a list
def avg_sentence_length(num_words, num_sentence):

  os.chdir('/content/drive/MyDrive/BlackCoffer/articles')
  avg_sent_len = []
  for i in range(len(os.listdir())):

    avg_sent_len.append(num_words[i]/num_sentence[i])

    i+=1 
  return avg_sent_len

In [ ]:
# subjectivity of each article
def subjectivity (positive_score, negative_score,number_of_words):
  subjectivity = []
  os.chdir('/content/drive/MyDrive/BlackCoffer/articles')
  for i in range(len(os.listdir())):
    subjectivity.append((positive_score[i] + negative_score[i])/ (number_of_words[i] + 0.000001))

    i+=1

  return subjectivity

In [ ]:
# calculate number of syllables in each word
def count_syllables(word):
  syllables = [element for element in [word[i] for i in range(len(word))] if element in ['a','e','i','o','u'] ]
  number_of_syllables = len(syllables)
  try:

    if word[-1] == 'e':
      number_of_syllables = number_of_syllables -1

    if word[-2:] == 'le' or word[-3:]=='les':
      if word[-3] not in ['a','e','i','o','u']:
        number_of_syllables = number_of_syllables + 1

    if word[-2:] == 'es' or 'ed':
      number_of_syllables = number_of_syllables -1
  
  except Exception as e:
    pass
   


  return len(syllables)
    

In [ ]:
# count number of complex words in each article
def count_complex_words(contents):
  complex_word_count={}
  os.chdir('/content/drive/MyDrive/BlackCoffer/articles')
  for i in range(len(contents)):
      complex_word_count[i] = 0
      for word in contents[i].split():
          if count_syllables(word) > 2:
              complex_word_count[i] += 1
      i+=1
  
  return complex_word_count
    

In [ ]:
 #percentage of complex words   
def complex_percentage(number_of_complex_words,number_of_words):
  percent_complex =[]

  os.chdir('/content/drive/MyDrive/BlackCoffer/articles')
  for i in range(len(os.listdir())):
    try:
      percent_complex.append((number_of_complex_words[i]/number_of_words[i])*100)
    except Exception as e:
      percent_complex.append(0)

    i+=1
  
  return percent_complex

In [ ]:
# fog index of each article
def fog_index(avg_sentence_length,percent_complex):
  Fog_Index =[]
  os.chdir('/content/drive/MyDrive/BlackCoffer/articles')

  for i in range(len(os.listdir())):
    p = 0.4 * (avg_sentence_length[i] + percent_complex[i])
    Fog_Index.append(p)
    i+=1
  
  return Fog_Index

In [ ]:
# number of syllable per word
def syllable_per_word(contents, number):
  syllable_per_word=[]

  for i in range(len(number)) :
    count = 0
    current = contents[i].split()
    for j in range(len(current)):
      syl = count_syllables(current[j])
      count= count+syl
      j+=1
    try:
      syllable_per_word.append(count/number[i])
    except Exception as e:
      syllable_per_word.append('0')     
    i+=1

  return syllable_per_word

In [ ]:
# average word length of each article
def avg_word_length(contents, number):
  avg_word_len=[]
  for i in range(len(number)):
    count = 0
    current = contents[i].split()
    for j in range(len(current)):
     count = count + len(current[j])
     j+=1
    try:
     avg_word_len.append(count/number[i])
    except Exception as e:
      avg_word_len.append('0') 
    i += 1

  return avg_word_len

### **CREATE THE OUTPUT FOLDER**

In [ ]:
def create_output_directory():
  if not os.path.exists("Output"):
      os.makedirs("Output")

In [ ]:
output_frame = pd.DataFrame(columns = ['URL_ID', 'URL', 'POSITIVE SCORE',
                                       'NEGATIVE SCORE','POLARITY SCORE',
                                       'SUBJECTIVITY SCORE','AVG SENTENCE LENGTH','PERCENTAGE OF COMPLEX WORDS',
                                       'FOG INDEX','AVG NUMBER OF WORDS PER SENTENCE','COMPLEX WORD COUNT',
                                       'WORD COUNT','SYLLABLE PER WORD','PERSONAL PRONOUNS','AVG WORD LENGTH'])

In [ ]:
output_frame

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH


In [ ]:
def fill_url_info():
  ids = list(data['URL_ID'])
  urls = list(data['URL'])
  output_frame['URL_ID'] = ids
  output_frame['URL'] = urls
  return output_frame

In [ ]:
fill_url_info()

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,37.0,https://insights.blackcoffer.com/ai-in-healthc...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,38.0,https://insights.blackcoffer.com/what-if-the-c...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,39.0,https://insights.blackcoffer.com/what-jobs-wil...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,40.0,https://insights.blackcoffer.com/will-machine-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,41.0,https://insights.blackcoffer.com/will-ai-repla...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,146.0,https://insights.blackcoffer.com/blockchain-fo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110,147.0,https://insights.blackcoffer.com/the-future-of...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111,148.0,https://insights.blackcoffer.com/big-data-anal...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112,149.0,https://insights.blackcoffer.com/business-anal...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def fill_score():
  contents = get_contents()
  l1=get_positive_score(contents)
  output_frame['POSITIVE SCORE'] = l1
  l2 =get_negative_score(contents)
  output_frame['NEGATIVE SCORE'] = l2
  output_frame['POLARITY SCORE'] = polarity(l1,l2)
  w = number_of_words(contents)
  s = num_sentences()
  output_frame['WORD COUNT'] = w
  output_frame['SUBJECTIVITY SCORE'] = subjectivity(l1,l2,w)
  output_frame['SYLLABLE PER WORD']= syllable_per_word(contents,w)
  output_frame['AVG WORD LENGTH'] = avg_word_length(contents, w)
  output_frame['AVG SENTENCE LENGTH'] = avg_sentence_length(w,s)
  output_frame['AVG NUMBER OF WORDS PER SENTENCE'] = output_frame['WORD COUNT']/s





In [ ]:
fill_score()

In [ ]:
output_frame

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,37.0,https://insights.blackcoffer.com/ai-in-healthc...,44,27,0.239437,0.068401,20.352941,NaN,NaN,20.352941,NaN,1038,2.805395,NaN,7.319846
1,38.0,https://insights.blackcoffer.com/what-if-the-c...,39,28,0.164179,0.110561,10.821429,NaN,NaN,10.821429,NaN,606,2.608911,NaN,6.747525
2,39.0,https://insights.blackcoffer.com/what-jobs-wil...,48,25,0.315068,0.084983,13.215385,NaN,NaN,13.215385,NaN,859,2.927823,NaN,7.536671
3,40.0,https://insights.blackcoffer.com/will-machine-...,35,22,0.228070,0.082729,10.439394,NaN,NaN,10.439394,NaN,689,2.615385,NaN,6.732946
4,41.0,https://insights.blackcoffer.com/will-ai-repla...,37,22,0.254237,0.071429,16.520000,NaN,NaN,16.520000,NaN,826,2.544794,NaN,6.881356
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,146.0,https://insights.blackcoffer.com/blockchain-fo...,19,26,-0.155556,0.093946,11.975000,NaN,NaN,11.975000,NaN,479,2.519833,NaN,7.08977
110,147.0,https://insights.blackcoffer.com/the-future-of...,21,11,0.312500,0.054701,15.394737,NaN,NaN,15.394737,NaN,585,2.584615,NaN,6.878632
111,148.0,https://insights.blackcoffer.com/big-data-anal...,18,28,-0.217391,0.078365,9.467742,NaN,NaN,9.467742,NaN,587,2.638842,NaN,6.88075
112,149.0,https://insights.blackcoffer.com/business-anal...,19,5,0.583333,0.083624,17.937500,NaN,NaN,17.937500,NaN,287,2.937282,NaN,7.867596


In [ ]:
def fill_complex_scores():
  contents = get_contents()
  w = number_of_words(contents)
  c = list(count_complex_words(contents).values())
  pc = complex_percentage(c,w)
  output_frame['COMPLEX WORD COUNT'] = c
  output_frame['PERCENTAGE OF COMPLEX WORDS'] = complex_percentage(c,w)
  output_frame['FOG INDEX'] = fog_index(output_frame['AVG SENTENCE LENGTH'],pc)
  output_frame['PERSONAL PRONOUNS'] =personal_pronouns()
  


In [ ]:
fill_complex_scores()

In [ ]:
type(contents.values())

dict_values

In [ ]:
output_frame

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,37.0,https://insights.blackcoffer.com/ai-in-healthc...,44,27,0.239437,0.068401,20.352941,57.225434,31.031350,20.352941,594,1038,2.805395,21,7.319846
1,38.0,https://insights.blackcoffer.com/what-if-the-c...,39,28,0.164179,0.110561,10.821429,47.359736,23.272466,10.821429,287,606,2.608911,33,6.747525
2,39.0,https://insights.blackcoffer.com/what-jobs-wil...,48,25,0.315068,0.084983,13.215385,60.884750,29.640054,13.215385,523,859,2.927823,20,7.536671
3,40.0,https://insights.blackcoffer.com/will-machine-...,35,22,0.228070,0.082729,10.439394,46.734398,22.869517,10.439394,322,689,2.615385,49,6.732946
4,41.0,https://insights.blackcoffer.com/will-ai-repla...,37,22,0.254237,0.071429,16.520000,47.820823,25.736329,16.520000,395,826,2.544794,46,6.881356
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,146.0,https://insights.blackcoffer.com/blockchain-fo...,19,26,-0.155556,0.093946,11.975000,43.215031,22.076013,11.975000,207,479,2.519833,15,7.08977
110,147.0,https://insights.blackcoffer.com/the-future-of...,21,11,0.312500,0.054701,15.394737,49.401709,25.918578,15.394737,289,585,2.584615,24,6.878632
111,148.0,https://insights.blackcoffer.com/big-data-anal...,18,28,-0.217391,0.078365,9.467742,55.025554,25.797318,9.467742,323,587,2.638842,9,6.88075
112,149.0,https://insights.blackcoffer.com/business-anal...,19,5,0.583333,0.083624,17.937500,60.278746,31.286498,17.937500,173,287,2.937282,6,7.867596


In [ ]:
#save the output data in the form of an excel file

In [ ]:
os.chdir('/content/drive/MyDrive/BlackCoffer/')

In [ ]:
create_output_directory()

In [ ]:
os.chdir('Output')

In [ ]:
output_frame.to_excel('output.xlsx', index=False)